# Setup

In [1]:
from pathlib import Path
import json
import re
import pdfplumber

PROJECT_DIR = Path(".")  # τρέξε notebook από το root του project
PDF_DIR = PROJECT_DIR / "data_raw"
TXT_DIR = PROJECT_DIR / "data_text"
OUT_DIR = PROJECT_DIR / "data_processed"
OUT_JSONL = OUT_DIR / "chunks.jsonl"

CHUNK_SIZE = 2200
OVERLAP = 250
MIN_CHUNK_CHARS = 200

print("PDF_DIR:", PDF_DIR.resolve(), "exists:", PDF_DIR.exists())
print("PDF count:", len(list(PDF_DIR.glob("*.pdf"))))


PDF_DIR: C:\Users\ssar\Desktop\RAG_notes\project\data_raw exists: True
PDF count: 10


# PDF → TXT

In [2]:
TXT_DIR.mkdir(parents=True, exist_ok=True)

for pdf_path in PDF_DIR.glob("*.pdf"):
    all_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text.append(text)

    output_txt = TXT_DIR / f"{pdf_path.stem}.txt"
    output_txt.write_text("\n\n".join(all_text), encoding="utf-8")
    print(f"✅ {pdf_path.name} -> {output_txt.name}")

print("🎉 PDF -> TXT done")


✅ Acorn Archimedes Guide.pdf -> Acorn Archimedes Guide.txt
✅ Acorn Atom Technical Manual.pdf -> Acorn Atom Technical Manual.txt
✅ Compaq AlphaServer DS10 466 MHz AlphaStation DS10 466 MHz QuickSpecs (2001).pdf -> Compaq AlphaServer DS10 466 MHz AlphaStation DS10 466 MHz QuickSpecs (2001).txt
✅ Compaq AlphaServer DS20 QuickSpecs (2000).pdf -> Compaq AlphaServer DS20 QuickSpecs (2000).txt
✅ Compaq AlphaServer GS80 Installation Guide (2000).pdf -> Compaq AlphaServer GS80 Installation Guide (2000).txt
✅ Compaq Tru64 UNIX 5.1 AdvFS Administration ARH96BTE (2000).pdf -> Compaq Tru64 UNIX 5.1 AdvFS Administration ARH96BTE (2000).txt
✅ Compaq Tru64 UNIX 5.1 Master Index ARH94BTE (2000).pdf -> Compaq Tru64 UNIX 5.1 Master Index ARH94BTE (2000).txt
✅ Compaq Tru64 UNIX 5.1 Writing Software for the International Market ARH9YBTE (2000).pdf -> Compaq Tru64 UNIX 5.1 Writing Software for the International Market ARH9YBTE (2000).txt
✅ HP OpenVMS 8.3-1H1 Guide to Creating OpenVMS Modular Procedures OVMS

# Cleaning + chunking funcs

In [3]:
def clean_text(s: str) -> str:
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    s = re.sub(r"\n{3,}", "\n\n", s)
    s = re.sub(r"[ \t]{2,}", " ", s)
    return s.strip()

def chunk_text(text: str, chunk_size: int = CHUNK_SIZE, overlap: int = OVERLAP):
    if overlap >= chunk_size:
        raise ValueError("overlap must be smaller than chunk_size")

    chunks = []
    start = 0
    n = len(text)

    while start < n:
        end = min(start + chunk_size, n)
        chunk = text[start:end].strip()
        if len(chunk) >= MIN_CHUNK_CHARS:
            chunks.append((start, end, chunk))
        start = end - overlap
        if start < 0:
            start = 0
        if end == n:
            break

    return chunks


In [4]:
# TXT → chunks.jsonl

In [5]:
OUT_DIR.mkdir(parents=True, exist_ok=True)

txt_files = sorted(TXT_DIR.glob("*.txt"))
print("TXT files:", len(txt_files))

rows = []
for txt_path in txt_files:
    doc_id = txt_path.stem
    raw = txt_path.read_text(encoding="utf-8", errors="ignore")
    text = clean_text(raw)

    for j, (start_char, end_char, chunk) in enumerate(chunk_text(text)):
        rows.append({
            "doc_id": doc_id,
            "source": str(txt_path),
            "chunk_id": f"{doc_id}_{j:05d}",
            "text": chunk,
            "start_char": start_char,
            "end_char": end_char
        })

with open(OUT_JSONL, "w", encoding="utf-8") as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("✅ Wrote:", OUT_JSONL)
print("Total chunks:", len(rows))


TXT files: 10
✅ Wrote: data_processed\chunks.jsonl
Total chunks: 976


# Sanity check

In [6]:
for r in rows[:3]:
    print("\n---", r["chunk_id"], "---")
    print(r["text"][:600])



--- Acorn Archimedes Guide_00000 ---
Displaying a menu
1. Point to the relevant Move the pointer over the window or icon
icon you want to work on.
For example, move the pointer over the
Applications disc 1 directory window.
2. Click on the middle The menu will be displayed.
mouse button
Note that the menu will change to apply to an
individual file in a directory window, if the
pointer is over the file name, or if you have
previously selected its icon by clicking on it.
Choosing from a menu
1. Select the menu item Move the pointer over the menu item you
you want want and click the select (LH) button.
The action requested will occu

--- Acorn Archimedes Guide_00001 ---
any part of the information contained in, nor the
product described in this Guide may be adapted or reproduced in any
material form except with the prior written approval of Acorn Computers
Limited.
The products described in this manual are subject to continuous development
and improvement. All information of a technical 